In [ ]:
import json
import logging
import os
import time
from datetime import datetime
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup
from minio import Minio

In [ ]:
minio_connection = ""

In [ ]:
# carregar para funcionar
try:
    minio_conn = json.loads(minio_connection)
except json.JSONDecodeError:
    with open('../variables/minio_connection.json', "r") as minio_connection_file:
        minio_conn = json.loads(minio_connection_file.read())

In [ ]:
endpoint_raw = minio_conn["endpoint"]
endpoint_sem_http = endpoint_raw.replace("http://", "").replace("https://", "")

s3_client = Minio(
    endpoint=endpoint_sem_http,
    access_key=minio_conn['access_key'],
    secret_key=minio_conn['key'],
    secure=endpoint_raw.startswith("https")
)

In [ ]:
# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# --- Variáveis do Projeto ---
url = "https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-revenda-e-de-distribuicao-combustiveis/serie-historica-do-levantamento-de-precos"
extensoes_arquivos = ['.xls', '.xlsx', '.zip']
bucket_name = "landing" 
s3_prefix = "anp/serie_levantamento_precos/" 

In [ ]:
def extrair_urls_arquivos(url_pagina, extensoes_permitidas):
    """
    Extrai URLs de arquivos de uma página web, filtrando por extensões e conteúdo.
    """
    logging.info(f"Acessando a URL: {url_pagina} para extrair links...")
    response = requests.get(url_pagina)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    links_de_arquivos = set() # Usamos um set para evitar duplicatas

    for link in soup.find_all("a", href=True):
        href = link["href"]
        texto = link.get_text(strip=True).lower()
        
        # Filtra por extensão e conteúdo "mensal"
        if any(href.lower().endswith(ext) for ext in extensoes_permitidas) and "mensal" in (href.lower() + texto):
            href_absoluto = urljoin(url_pagina, href)
            links_de_arquivos.add(href_absoluto)

    logging.info(f"Encontrados {len(links_de_arquivos)} links para download.")
    return list(links_de_arquivos)

In [ ]:
def baixar_arquivos_com_metadados(links, pasta_destino="download"):
    """
    Baixa arquivos de uma lista de links e retorna metadados.
    """
    os.makedirs(pasta_destino, exist_ok=True)
    registros = []
    
    logging.info(f"Iniciando o download de {len(links)} arquivos para '{pasta_destino}'...")

    for link in links:
        nome_arquivo = link.split("/")[-1]
        caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)
        data_download = datetime.now().isoformat()

        try:
            resposta = requests.get(link)
            if resposta.status_code == 200:
                with open(caminho_arquivo, "wb") as f:
                    f.write(resposta.content)
                logging.info(f"Baixado: {nome_arquivo}")

                registros.append({
                    "nome_arquivo": nome_arquivo,
                    "link_origem": link,
                    "caminho_local": caminho_arquivo,
                    "data_download": data_download
                })
            else:
                logging.warning(f"Erro ao baixar: {link} (Status {resposta.status_code})")
        except Exception as e:
            logging.error(f"Erro ao baixar {link} -> {e}", exc_info=True)

        time.sleep(0.5) # Pausa para não sobrecarregar o servidor
    
    logging.info(f"Download finalizado. {len(registros)} arquivos baixados com sucesso.")
    return registros

In [ ]:
def upload_files_to_minio(registros_baixados):
    """
    Faz o upload de uma lista de arquivos locais para o MinIO.
    """
    logging.info(f"\nIniciando o upload de {len(registros_baixados)} arquivos para o bucket '{bucket_name}'...")
    
    for registro in registros_baixados:
        caminho_local = registro["caminho_local"]
        nome_arquivo = registro["nome_arquivo"]
        s3_caminho_destino = s3_prefix + nome_arquivo

        try:
            s3_client.fput_object(bucket_name, s3_caminho_destino, caminho_local)
            logging.info(f"Upload bem-sucedido: {caminho_local} -> s3a://{bucket_name}/{s3_caminho_destino}")
        except Exception as e:
            logging.error(f"Erro ao fazer o upload do arquivo {nome_arquivo}: {e}")

    logging.info("Upload de arquivos finalizado.")

In [ ]:
# 1. Extrair URLs
links_de_arquivos = extrair_urls_arquivos(url, extensoes_arquivos)

# 2. Baixar arquivos
if links_de_arquivos:
    registros_baixados = baixar_arquivos_com_metadados(links_de_arquivos)

    # 3. Fazer o upload e, em seguida, excluir
    if registros_baixados:
        try:
            upload_files_to_minio(registros_baixados)
            logging.info("Processo de extração, download e upload finalizado com sucesso.")
        finally:
            for registro in registros_baixados:
                os.remove(registro['caminho_local'])
                logging.info(f"Arquivo temporário excluído: {registro['caminho_local']}")
    else:
        logging.warning("Nenhum arquivo foi baixado. O processo de upload não foi iniciado.")
else:
    logging.warning("Nenhum link de arquivo encontrado. O processo foi encerrado.")